<a href="https://colab.research.google.com/github/melinadwisafitri/mental-care/blob/machine-learning/Chatbot%20(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import json
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [ ]:
data_url = 'https://raw.githubusercontent.com/melinadwisafitri/mental-care/machine-learning/dataset/data.json'
urllib.request.urlretrieve(data_url, 'data.json')

('data.json', <http.client.HTTPMessage at 0x7f981f378510>)

In [ ]:
with open('data.json', 'r') as f:
  data = json.load(f)

In [ ]:
questions = []
answers = []

for item in data['mental_healths']:
  questions.append(item['question'])
  answers.append(item['answer'])

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
training_questions = questions[0:training_size]
testing_questions = questions[training_size:]
training_answers = answers[0:training_size]
testing_answers = answers[training_size:]

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_questions)

word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_question(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in question])
def decode_answer(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in answer])

training_sequences = tokenizer.texts_to_sequences(training_questions)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

training_padded = np.array(training_padded)
training_answers = np.array(training_answers)

testing_sequences = tokenizer.texts_to_sequences(testing_questions)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_padded = np.array(testing_padded)
testing_answers = np.array(testing_answers)

In [15]:
 model = tf.keras.Sequential([
      tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
      tf.keras.layers.LSTM(64),
      tf.keras.layers.Dense(128, input_shape=(len(training_padded[0]),), activation='relu'),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(len(training_answers[0]), activation='softmax')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(training_padded,
          training_answers,
          epochs=10,
          validation_data=(testing_padded, testing_answers))

ValueError: ignored